## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Sun Sep  1 09:36:53 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Install YOLOv8

⚠️ YOLOv8 is still under heavy development. Breaking changes are being introduced almost weekly. We strive to make our YOLOv8 notebooks work with the latest version of the library. Last tests took place on **03.01.2024** with version **YOLOv8.0.196**.

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [2]:
!pip install --upgrade ray
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 22.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: ray
    Found existing installation: ray 2.24.0
    Uninstalling ray-2.24.0:
      Successfully uninstalled ray-2.24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 559.3 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 5.3 MB/s eta 0:00:00a 0:00:01


In [3]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.85 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5846.1/8062.4 GB disk)


In [4]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

In [5]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="********************")
project = rf.workspace("nishant-raghuwanshi-deep-learning").project("traffic-sign-guy19-v6hko")
version = project.version(1)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 875.9 kB/s eta 0:00:00a 0:00:01
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.85, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to traffic-sign-1 in yolov8:: 100%|██████████| 8238/8238 [00:01<00:00, 7580.00it/s]


## Custom Training

In [6]:
!yolo task=detect mode=train model=yolov8m.yaml data=/kaggle/input/yaml-file/data.yaml epochs=80 imgsz=800 plots=False

Ultralytics YOLOv8.2.85 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.yaml, data=/kaggle/input/yaml-file/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

In [8]:
!ls /kaggle/working/runs/detect/train

args.yaml					   results.csv
events.out.tfevents.1725183710.ef3a87c5bce2.110.0  weights


## Validate Custom Model

In [11]:
%cd {HOME}

!yolo task=detect mode=val model=/kaggle/working/runs/detect/train/weights/best.pt data= /kaggle/input/yaml-file/data.yaml

[Errno 2] No such file or directory: '{HOME}'
/kaggle/working
Ultralytics YOLOv8.2.85 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m summary (fused): 218 layers, 25,882,606 parameters, 0 gradients, 78.9 GFLOPs
val: Scanning /kaggle/working/traffic-sign-1/valid/labels.cache... 763 images, 3
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        763        859      0.788      0.674      0.769      0.526
                     -          1          1          0          0          0          0
                     0        100        178      0.791      0.669      0.734      0.357
         Barrier Ahead         15         15      0.989          1      0.995      0.876
                Cattle          3          3      0.704      0.667       0.83      0.749
               Caution          1          1          0          0     0.0226     0.0158
        Cycle Crossing         11         11      0.858          1      0.96

## Inference with Custom Model

In [12]:
%cd {HOME}
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train/weights/best.pt conf=0.25 source=/kaggle/working/traffic-sign-1/test/images save=True

[Errno 2] No such file or directory: '{HOME}'
/kaggle/working
Ultralytics YOLOv8.2.85 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m summary (fused): 218 layers, 25,882,606 parameters, 0 gradients, 78.9 GFLOPs

image 1/807 /kaggle/working/traffic-sign-1/test/images/BA-10-_jpg.rf.f28da62af45cd14831a267587f9dc38d.jpg: 800x800 1 Barrier Ahead, 59.2ms
image 2/807 /kaggle/working/traffic-sign-1/test/images/BA-11-_png.rf.40e7ecd4d042d5d6b9109dc2a7f44f78.jpg: 800x800 1 Barrier Ahead, 59.3ms
image 3/807 /kaggle/working/traffic-sign-1/test/images/BA-12-_jpg.rf.04a65389092f8e771da78bbc2a733891.jpg: 800x800 1 Dangerous Dip, 1 Give Way, 55.8ms
image 4/807 /kaggle/working/traffic-sign-1/test/images/BA-13-_png.rf.be372cad9968de45159eb8100db88cc7.jpg: 800x800 1 Barrier Ahead, 47.9ms
image 5/807 /kaggle/working/traffic-sign-1/test/images/BA-16-_png.rf.2c212cc7c42e25e96d66a9b32a30a62a.jpg: 800x800 1 Barrier Ahead, 48.0ms
image 6/807 /kaggle/working/traffic-sign-1/test/images/BA-20-_pn

## Deploy model on Roboflow

Once you have finished training your YOLOv8 model, you’ll have a set of trained weights ready for use. These weights will be in the `/runs/detect/train/weights/best.pt` folder of your project. You can upload your model weights to Roboflow Deploy to use your trained weights on our infinitely scalable infrastructure.

The `.deploy()` function in the [Roboflow pip package](https://docs.roboflow.com/python) now supports uploading YOLOv8 weights.

To upload model weights, add the following code to the “Inference with Custom Model” section in the aforementioned notebook:

In [14]:
project.version(dataset.version).deploy(model_type="yolov8", model_path="/kaggle/working/runs/detect/train/")

Dependency ultralytics==8.0.196 is required but found version=8.2.85, to fix: `pip install ultralytics==8.0.196`


Would you like to continue with the wrong version of ultralytics? y/n:  y


View the status of your deployment at: https://app.roboflow.com/nishant-raghuwanshi-deep-learning/traffic-sign-guy19-v6hko/1
Share your model with the world at: https://universe.roboflow.com/nishant-raghuwanshi-deep-learning/traffic-sign-guy19-v6hko/model/1
